In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import math
from statsmodels.tsa.seasonal import seasonal_decompose

In [6]:
df = pd.read_csv("../../data/preprocessed/base.csv", sep=",")
df['periodo_dt'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')
df.shape

(2945818, 14)

#### Filtrado de Productos con Ventas Estables y Suficiente Historial

Este código selecciona del dataset original (`df`) los productos que cumplen con dos condiciones:

1️⃣ Han tenido ventas positivas en al menos 24 meses (criterio de "historial suficiente").

2️⃣ Están presentes en el historial de ventas durante un número total de meses especificado (en este caso, se calcula el total de meses únicos en el dataset).

In [30]:
# Asegurar ventas positivas
ventas = df[df['tn'] > 0].copy()

# Calcular cuántos meses únicos con ventas tuvo cada producto
historial = (
    ventas.groupby('product_id')['periodo']
    .nunique()
    .reset_index(name='meses_con_ventas')
)

# Definir el total de meses del histórico disponible
total_meses = df['periodo'].nunique()

# Filtrar productos que:
# - tienen más de 3 meses con ventas
# - tienen el total de meses esperados (por ejemplo 36)
productos_estables = historial[historial['meses_con_ventas'] >= 24]['product_id'].tolist()

# Filtrar el DataFrame original para esos productos
df_productos_estables = df[df['product_id'].isin(productos_estables)]

print(f"🔎 Se encontraron {len(productos_estables)} productos estables (vendidos todos los meses del histórico).")


🔎 Se encontraron 780 productos estables (vendidos todos los meses del histórico).


#### Descomposición de Series Temporales por Producto y Combinaciones de Categorías

Este código visualiza y analiza las series temporales de ventas (tn) para cada producto (product_id), organizadas según combinaciones de categorías (cat1, cat2, cat3).

In [ ]:
def descomposicion_serie_productos_por_cat(df, cat1=None, cat2=None):
    """
    Aplica seasonal_decompose o grafica la serie por cada product_id
    dentro de cada combinación cat1, cat2 y cat3.
    """

    # Filtrar combinaciones válidas
    subset = df[(df['cat1'] == cat1) & (df['cat2'] == cat2) & (df['tn'] > 0)].copy()
    if subset.empty:
        return

    subset['periodo_dt'] = pd.to_datetime(subset['periodo'].astype(str), format='%Y%m')

    # Recorrer cada cat3 dentro del cat1 y cat2 actual
    for cat3 in subset['cat3'].dropna().unique():
        ventas = subset[subset['cat3'] == cat3].copy()

        # Agrupar por producto y mes
        ventas_agrupadas = (
            ventas.groupby(['product_id', 'periodo_dt'])['tn']
            .sum()
            .reset_index()
            .sort_values(['product_id', 'periodo_dt'])
        )

        product_ids = ventas_agrupadas['product_id'].unique()
        n = len(product_ids)
        n_cols = 2
        n_rows = math.ceil(n / n_cols)

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
        axes = axes.flatten()

        for i, product_id in enumerate(product_ids):
            serie = (
                ventas_agrupadas[ventas_agrupadas['product_id'] == product_id]
                .set_index('periodo_dt')['tn']
                .asfreq('MS')
            )

            ax = axes[i]

            try:
                if serie.notna().sum() >= 12:
                    resultado = seasonal_decompose(serie, model='additive', period=12)
                    ax.plot(serie.index, serie.values, label='Serie original', alpha=0.5, color='black')
                    ax.plot(resultado.trend.index, resultado.trend.values, label='Tendencia', color='blue')
                    ax.plot(resultado.seasonal.index, resultado.seasonal.values, label='Estacionalidad', linestyle='--', color='green')
                    ax.set_title(f"{cat1} - {cat2} - {cat3} - ID {product_id}")
                else:
                    ax.plot(serie.index, serie.values, marker='o', linestyle='-', color='gray')
                    ax.set_title(f"{cat1} - {cat2} - {cat3} - ID {product_id} (sin descomposición)")

                # Línea vertical en enero
                for fecha in serie.index:
                    if fecha.month == 1:
                        ax.axvline(x=fecha, color='gray', linestyle=':', linewidth=2)

                ax.set_xlabel("Periodo")
                ax.set_ylabel("Toneladas vendidas")
                ax.tick_params(axis='x', rotation=45)
                ax.legend()

            except Exception as e:
                print(f"Error en producto {product_id}: {e}")
                fig.delaxes(ax)

        # Eliminar ejes vacíos
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()




# Obtener combinaciones únicas de cat1 y cat2, eliminando valores nulos en cat1
combinaciones = df[['cat1', 'cat2']].dropna(subset=['cat1']).drop_duplicates()

# Recorrer cada combinación y aplicar el gráfico
for _, fila in combinaciones.iterrows():
    cat1 = fila['cat1']
    cat2 = fila['cat2']
    
    print(f"\n📈 Generando gráfico para: cat1 = {cat1} | cat2 = {cat2}")
    descomposicion_serie_productos_por_cat(df, cat1=cat1, cat2=cat2)

#### Visualización de Series Temporales por Categoría con Tendencia Lineal

Este código genera gráficos que muestran la evolución de ventas (tn) a lo largo del tiempo para cada producto (product_id), organizados por combinaciones de categorías (cat1, cat2, cat3).

In [ ]:
def graficar_series_con_tendencia(df, cat1, cat2):
    """
    Grafica la serie original y su tendencia lineal para cada producto (por cat1, cat2, cat3).
    Incluye líneas punteadas en enero.
    """
    subset = df[(df['cat1'] == cat1) & (df['cat2'] == cat2) & (df['tn'] > 0)].copy()
    if subset.empty:
        print(f"No hay datos para cat1={cat1} y cat2={cat2}")
        return

    subset['periodo_dt'] = pd.to_datetime(subset['periodo'].astype(str), format='%Y%m')

    for cat3 in subset['cat3'].dropna().unique():
        df_cat3 = subset[subset['cat3'] == cat3]
        product_ids = df_cat3['product_id'].unique()
        n = len(product_ids)
        n_cols = 2
        n_rows = math.ceil(n / n_cols)

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
        axes = axes.flatten()

        for i, product_id in enumerate(product_ids):
            serie = (
                df_cat3[df_cat3['product_id'] == product_id]
                .groupby('periodo_dt')['tn']
                .sum()
                .sort_index()
            )

            # Asegurar continuidad de fechas
            fechas = pd.date_range(start=serie.index.min(), end=serie.index.max(), freq='MS')
            serie = serie.reindex(fechas, fill_value=0)

            ax = axes[i]
            ax.plot(serie.index, serie.values, marker='o', linestyle='-', color='steelblue', label='Serie original')

            # Calcular línea de tendencia
            x = np.arange(len(serie))
            y = serie.values
            coef = np.polyfit(x, y, 1)  # Ajuste lineal
            tendencia = np.polyval(coef, x)
            ax.plot(serie.index, tendencia, color='crimson', linestyle='--', linewidth=2, label='Tendencia lineal')

            # Personalización
            ax.set_title(f"{cat1} - {cat2} - {cat3} - ID {product_id}")
            ax.set_xlabel("Periodo")
            ax.set_ylabel("Toneladas")
            ax.tick_params(axis='x', rotation=45)

            # Líneas verticales en enero
            for fecha in serie.index:
                if fecha.month == 1:
                    ax.axvline(x=fecha, color='gray', linestyle=':', linewidth=1)

            ax.legend()

        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()

# Obtener combinaciones únicas de cat1 y cat2, eliminando valores nulos en cat1
combinaciones = df[['cat1', 'cat2']].dropna(subset=['cat1']).drop_duplicates()

# Recorrer cada combinación y aplicar el gráfico
for _, fila in combinaciones.iterrows():
    cat1 = fila['cat1']
    cat2 = fila['cat2']
    
    print(f"\n📈 Generando gráfico para: cat1 = {cat1} | cat2 = {cat2}")
    graficar_series_con_tendencia(df, cat1=cat1, cat2=cat2)

In [ ]:
def graficar_series_con_tendencia_suavizada(df, cat1, cat2):
    """
    Grafica la serie original de cada producto (cat1-cat2-cat3) con su tendencia suavizada
    usando seasonal_decompose. Agrega líneas punteadas en cada enero.
    """
    subset = df[(df['cat1'] == cat1) & (df['cat2'] == cat2) & (df['tn'] > 0)].copy()
    if subset.empty:
        print(f"No hay datos para cat1={cat1} y cat2={cat2}")
        return

    subset['periodo_dt'] = pd.to_datetime(subset['periodo'].astype(str), format='%Y%m')

    for cat3 in subset['cat3'].dropna().unique():
        df_cat3 = subset[subset['cat3'] == cat3]
        product_ids = df_cat3['product_id'].unique()
        n = len(product_ids)
        n_cols = 2
        n_rows = math.ceil(n / n_cols)

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5 * n_rows))
        axes = axes.flatten()

        for i, product_id in enumerate(product_ids):
            serie = (
                df_cat3[df_cat3['product_id'] == product_id]
                .groupby('periodo_dt')['tn']
                .sum()
                .sort_index()
            )

            # Crear rango completo mensual
            fechas_completas = pd.date_range(start=serie.index.min(), end=serie.index.max(), freq='MS')
            serie = serie.reindex(fechas_completas, fill_value=0)

            ax = axes[i]
            ax.plot(serie.index, serie.values, marker='o', linestyle='-', color='steelblue', label='Serie original')

            # Graficar la tendencia suavizada si hay suficiente info
            if serie.notna().sum() >= 24:
                resultado = seasonal_decompose(serie, model='additive', period=12)
                tendencia = resultado.trend
                ax.plot(tendencia.index, tendencia.values, color='crimson', linestyle='--', linewidth=2, label='Tendencia suavizada')

            ax.set_title(f"{cat1} - {cat2} - {cat3} - ID {product_id}")
            ax.set_xlabel("Periodo")
            ax.set_ylabel("Toneladas")
            ax.tick_params(axis='x', rotation=45)

            # Línea vertical en enero
            for fecha in serie.index:
                if fecha.month == 1:
                    ax.axvline(x=fecha, color='gray', linestyle=':', linewidth=1)

            ax.legend()

        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()

# Recorrer cada combinación y aplicar el gráfico
for _, fila in combinaciones.iterrows():
    cat1 = fila['cat1']
    cat2 = fila['cat2']
    
    print(f"\n📈 Generando gráfico para: cat1 = {cat1} | cat2 = {cat2}")
    graficar_series_con_tendencia_suavizada(df, cat1=cat1, cat2=cat2)